In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px


C:\Users\Ronit Kataria\AppData\Local\Temp\ipykernel_7712\1789447527.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Ronit Kataria\AppData\Local\Temp\ipykernel_7712\1789447527.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
df = pd.read_csv("loan_small.csv")

In [3]:
df["emp_length"] = df["emp_length"].str.replace("years", " ")
df["emp_length"] = df["emp_length"].str.replace("year", " ")
df["emp_length"] = df["emp_length"].str.replace("10+ ", "11", regex = False)
df["emp_length"] = df["emp_length"].str.replace("< 1", "0.5", regex = False)
df["emp_length"] = df["emp_length"].astype(float)
df["loan_status"] = df["loan_status"].str.replace("Fully Paid", "good")
df["loan_status"] = df["loan_status"].str.replace("Charged Off", "bad")
df["loan_status"] = df["loan_status"].str.replace("Current", "good")
df["loan_status"] = df["loan_status"].str.replace("Late (31-120 days)", "bad", regex = False)
df["loan_status"] = df["loan_status"].str.replace("Default", "good")

In [4]:
df["term"] = df["term"].str.replace("months", " ").astype(int)

In [5]:
# I have plotted the variation of installment n the form of a box plot where the grade and the home ownership can be decided using the interactive elements on the screen
fig = px.box(df, x='home_ownership', y='installment', color="grade")
app = dash.Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#E0B8C0'
}

app.layout = html.Div(children=[

    html.H1(
        children='Visualization of Loan Data',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='This Visualisation uses a Box Plot to visualize the variation of installments as a box plot on our particular selection of home_owner ship and the grade ', style={
        'color': colors['text']
    }),

    # Drop down for deciding the grade
    dcc.Dropdown(
        id='gradeDropdown',
        options=[ {'label': i, 'value': i} for i in df['grade'].unique()],
        value='A'
    ),

    # This drop down helps us select the home owner ship status that we want to see the graph for 

    dcc.Dropdown(
        id='homeDropdown',
        options=[ {'label': i, 'value': i} for i in df['home_ownership'].unique()],
        value='RENT'
    ),

    dcc.Graph(
        id='boxPlot',
        figure=fig
    )
])

@app.callback(
    dash.dependencies.Output(component_id='boxPlot', component_property='figure'),
    [dash.dependencies.Input(component_id='gradeDropdown', component_property='value'),
    dash.dependencies.Input(component_id='homeDropdown', component_property='value')]
)
def update_plot(grade,home_ownership):

    filtered_df = df[(df['grade']==grade) & (df["home_ownership"]==home_ownership)]
    fig = px.box(filtered_df, x="home_ownership", y="installment", color="grade")

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
